In [3]:
# Graph Builder
from collections import namedtuple
from pandas.api.types import is_numeric_dtype, is_categorical_dtype, is_categorical
import dgl
import torch

In [4]:
#from "module" import * 일때 *에 무엇을 포함 시킬 것 인지 
# __all__ = ['PandasGraphBulder']
import os 
import pandas as pd
import numpy as np
class graph_builder(object):

    def __init__(self):
        super(graph_builder,self).__init__()

    def load_data(self, path, User_feature_path = None, Item_feature_path = None):
        """ 
        Load the data from file path and build the graph
        Parameter:
        train_path : str => file path for train data set
        test_path : str => file path for test data set
        User_feature_path : str , default = None => file path for user data 
        Item_feature_path : str , default = None => file path for item data 
        Return:
        Without Feature : rating graph as g
        Without User Feature : user_feature, dgl graph as g
        Without Item Feature : item_feature, dgl graph as g
        With Feature : item_feature, user_feature, dgl graph as g
        """
        if os.path.exists(path):
            self.edges = pd.read_table(path, names=["user","item","timestamp"], sep=" ",dtype={'user':np.int64,'item':np.int64})
            self.num_user_node = len(self.edges['user'].unique())
            self.num_item_node = len(self.edges['item'].unique())
            user_item_src = self.edges['user']
            user_item_dst = self.edges['item']

            print(f"User node : {self.num_user_node}\n \
                  Item node : {self.num_item_node}\n \
                  edges : {len(self.edges)}")
        else:
            FileNotFoundError(f"There are No train data in {path}")

        if User_feature_path != None:
            if os.path.exists(User_feature_path):
                self.user_feature = pd.read_table(User_feature_path)
            else:
                FileNotFoundError(f"There are No user feature data in {User_feature_path}")
        else:
            self.user_feature = None
        if Item_feature_path != None:
            if os.path.exists(Item_feature_path):
                self.item_feature = pd.read_table(Item_feature_path)
            else:
                FileNotFoundError(f"There are No user feature data in {Item_feature_path}")
        else:
            self.user_feature = None

        data_dict = {
            ("user", "ui", "item"): (user_item_src, user_item_dst),
            ("item", "iu", "user"): (user_item_dst, user_item_src),
        }
        num_nodes_dict = { "user" : self.num_user_node, "item" : self.num_item_node}

        self.g = dgl.heterograph(data_dict, num_nodes_dict)
        
        return

    def sample(self):
        

    
    

In [ ]:
import torch.nn as nn

class MGCCF(nn.Module):
    def __init__(self, data, emb_size, n_layers):
        super(MGCCF, self).__init__()
        self.n_user = data.num_user_node
        self.n_item = data.num_item_node
        self.emb_size = emb_size
        # 512
        self.layers = n_layers
        # [128, 64 or 128]


    def init_weight(self):
        # xavier init
        initializer = nn.init.xavier_uniform_

        self.embedding_dict = nn.ParameterDict({
            'user_emb': nn.Parameter(initializer(torch.empty(self.n_user,
                                                 self.emb_size))),
            'item_emb': nn.Parameter(initializer(torch.empty(self.n_item,
                                                 self.emb_size)))
        })
        
        #init the paramenter
        self.user_weight_dict = nn.ParameterDict()
        layers = [self.emb_size] + self.layers
        for k in range(len(self.layers)):
            self.user_weight_dict.update({'W_gc_%d'%k: nn.Parameter(initializer(torch.empty(layers[k],
                                                                      layers[k+1])))})
            # Aggregation layer weight(Q)
            self.user_weight_dict.update({'W_ag_%d'%k: nn.Parameter(initializer(torch.empty(layers[k],
                                                                      layers[k+1])))})
        
        self.item_weight_dict = nn.ParameterDict()
        layers = [self.emb_size] + self.layers
        for k in range(len(self.layers)):
            self.item_weight_dict.update({'W_gc_%d'%k: nn.Parameter(initializer(torch.empty(layers[k],
                                                                      layers[k+1])))})
            # Aggregation layer weight(Q)
            self.item_weight_dict.update({'W_ag_%d'%k: nn.Parameter(initializer(torch.empty(layers[k],
                                                                      layers[k+1])))})
        
    def information_fusion(hidden, z, s):
        return hidden + z + s
    

    def create_bpr_loss(self, users, pos_items, neg_items):
        pos_scores = torch.sum(torch.mul(users, pos_items), axis=1)
        neg_scores = torch.sum(torch.mul(users, neg_items), axis=1)

        maxi = nn.LogSigmoid()(pos_scores - neg_scores)

        mf_loss = -1 * torch.mean(maxi)

        # cul regularizer
        regularizer = (torch.norm(users) ** 2
                       + torch.norm(pos_items) ** 2
                       + torch.norm(neg_items) ** 2) / 2
        emb_loss = self.decay * regularizer / self.batch_size

        return mf_loss + emb_loss, mf_loss, emb_loss
    
    
    def forward():
        # User Embedding Gcn

        # Item Embedding Gcn

        # Skip-Connection

        # MGE

        # Information Fusion

        # 